## The IMU Noise Model

The IMU measurement model used in Kalibr contains two types of sensor errors: $n$, an additive noise term that fluctuates very rapidly ("white noise"), and $b$, a slowly varying sensor bias. The angular rate measurement $\tilde\omega$ (for one single axis of the gyro, in this case) is therefore written as:

$$\tilde\omega(t)=\omega(t)+b(t)+n(t)$$

The same model is independently used to model all three sensor axes. The same model (with different parameters, as we will later see) is also used to model the accelerometer measurement errors (on each axis independently). This model is tractable and often used to model inertial sensors [1], [2].



### Additive "White Noise"

The rapid fluctuations in the sensor signal are modelled heuristically with a zero-mean, independent, continuous-time white Gaussian noise process $n(t)$ of strength $\sigma_g$:

$$E[n(t)]\equiv0$$

$$E[n(t_1)n(t_2)]=\sigma_g^2\delta(t_1-t_2)$$

In other words, the higher $\sigma_g$ is, the more "noisy" your gyro measurements. The parameters `gyroscope_noise_density` and `accelerometer_noise_density` in the [YAML file](yaml-formats) specify exactly these noise strengths $\sigma_g$ (gyro) and $\sigma_a$ (accel) for the continuous-time model.

This process can be simulated in **discrete-time** as follows:

$$n_d[k]=\sigma_{g_d}w[k]$$

with

$$w[k]\sim\mathcal{N}(0,1)$$

$$\sigma_{g_d}=\sigma_g\frac{1}{\sqrt{\Delta t}}$$

where $\Delta t$ is the sampling time. This is identical to the discrete-time implementation within Kalibr.

_Note_: This assumes that the noise was filtered with an ideal low-pass filter that filters noise above $f=\frac{1}{2\Delta t}$ (in other words, an ideal decimation stage). This may or may not be the case, depending on your sensor settings. If you simply "subsample" your gyro or accel, you are not allowed to scale your "white noise density" in that way.

How you can determine this parameter for your particular IMU is explained below.



### Bias
IMU biases are systematic errors that can be caused by various factors like temperature changes, manufacturing imperfections, or aging. 
In Kalibr, slow variations in the sensor bias are modelled with a "Brownian motion" process, also termed a "Wiener process", or "random walk" in discrete-time. Formally, this process is generated by integrating "white noise" of strength $\sigma_{b_g}$ (gyro) or $\sigma_{b_a}$ (accel):


$$\dot{b_g}(t)=\sigma_{b_g}w(t)$$


The formula $\dot{b_g}(t)=\sigma_{b_g}w(t)$ represents the rate of change of the gyroscope bias $b_g$ with respect to time $t$. Here:
- $\dot{b_g}(t)$ denotes the derivative of the bias $b_g$ at time $t$.
- $\sigma_{b_g}$ is the standard deviation of the gyroscope bias noise.
- $w(t)$ is a white noise process at time $t$.

This equation models the gyroscope bias as a stochastic process, where the rate of change of the bias is driven by a white noise process scaled by a standard deviation parameter $\sigma_{b_g}$.


where $w$ is "white noise" of unit strength.

The parameters `gyroscope_random_walk` and `accelerometer_random_walk` in the [YAML file](yaml-formats) specify these noise strengths $\sigma_{b_g}$ and $\sigma_{b_a}$. The higher the bias variations in your gyro or accel are, the higher these parameters need to be set.

This process can be simulated in discrete-time as follows:

$$b_d[k]=b_d[k-1]+\sigma_{bgd} w[k]$$

with

$$w[k]\sim\mathcal{N}(0,1)$$

$$\sigma_{bgd}=\sigma_{b_g}\sqrt{\Delta t}$$

This corresponds to the implementation in Kalibr.

The formula following the mentioned line models the bias of an IMU sensor as a random walk process, by integrating a white noise. In this formula:
- $ b[k] $ represents the bias at time $ k $.
- $ b[k-1] $ is the bias at the previous time step $ k-1 $.
- $ n_w[k] $ denotes white noise at time $ k $.
- $ \sigma_{b_w} $ is the white noise strength, determining the rate at which the bias changes over time.

This model suggests that the bias at each time step is the sum of the bias at the previous time step and a white noise term, which is scaled by the white noise strength $ \sigma_{b_w} $.



### The Noise Model Parameters in Kalibr

In Kalibr, the sensor errors are modelled for each sensor axis independently. Unless you are using inertial sensors whose axes have very different noise properties, this is legitimate. Internally, Kalibr will therefore model the "white" noise processes as follows:

$$E[\mathbf{n}(t)]=\mathbf{0}_{3\times1}$$

$$E[\mathbf{n}(t_1)\mathbf{n}^T(t_2)]=\sigma_g^2\mathbf{I}_{3\times3}\delta(t_1-t_2)$$

The bias variations ("random walks") are also modelled independently on each sensor axis. Table 1 summarizes all the parameters, and links them to the entries that you can specify in the [YAML file](yaml-formats). The units of the "sigmas" can also be found in [3].




* * *

Table 1: Summary of the IMU noise model parameters as they can be specified in the [IMU configuration YAML file](yaml-formats) of Kalibr.
A discussion of these units can be found [in this thread](https://github.com/ethz-asl/kalibr/issues/354#issuecomment-979934812) for those interested.

Parameter | YAML element | Symbol | Units
--- | --- | --- | ---
Gyroscope "white noise" | `gyroscope_noise_density` | $\sigma_{g}$ | $\frac{rad}{s}\frac{1}{\sqrt{Hz}}$
Accelerometer "white noise" | `accelerometer_noise_density` | $\sigma_{a}$ | $\frac{m}{s^2}\frac{1}{\sqrt{Hz}}$
Gyroscope "random walk" | `gyroscope_random_walk` | $\sigma_{b_g}$ | $\frac{rad}{s^2}\frac{1}{\sqrt{Hz}}$
Accelerometer "random walk" | `accelerometer_random_walk` | $\sigma_{b_a}$ | $\frac{m}{s^3}\frac{1}{\sqrt{Hz}}$
IMU sampling rate | `update_rate` | $\frac{1}{\Delta t}$ | $Hz$




## How to Obtain the Parameters for your IMU

This section describes how you can obtain the Kalibr IMU noise model parameters for your particular IMU. While there are many methods available, we focus on how to get the parameters from the datasheet or using the "Allan standard deviation".

### From the Datasheet of the IMU

**White Noise Terms:** The parameters for the "white noise" processes ($\sigma_g$, $\sigma_a$) are often specified in the datasheet of the sensor manufacturer. A bit misleading, they are commonly denoted as **angular random walk** in case of the gyro, and **velocity random walk** for the accel. The name comes from the fact that if this white noise on rate or acceleration is integrated (in the navigation equations), it becomes a "random walk" on the angle or the velocity.

Other manufacturers specify it directly as **rate noise density**, **acceleration noise density**, or simply **noise density**. The name comes from the fact that $\sigma_g^2$ corresponds to the power spectral _density_ of $n$. Using the discrete-time implementation outlined above, you can check if you interpreted the values in the datasheet correctly.


**Bias Terms:** In contrast to the "white noise sigmas", $\sigma_{b_g}$ and $\sigma_{b_a}$ are rarely directly specified in the datasheet. The reason is that in practice, the bias does not truly behave like a "random walk" for longer integration times. Often, the so-called **in-run bias (in)stability** is specified instead. This sensor parameter indicates (approximately) the accuracy with which the bias can be determined (if a random process is the sum of "white noise" and a "random walk" bias, the bias can not be estimated with arbitrarily low uncertainty at any point in time). In combination with the strength of the "white noise", however, one can often use the in-run bias stability (the lowest point in the Allan standard deviation, see below) to determine reasonable values for $\sigma_{b_g}$ and $\sigma_{b_a}$ (assuming that the noise is dominated by "white noise" and a "random walk").

### From the Allan standard deviation (AD)

While many (parametric and non-parametric) methods have been proposed to (automatically) determine the noise model parameters from samples, deriving the parameters from an Allan standard deviation plot is probably the most common and standardized procedure. [1] derives the AD for different random processes, including "white noise" (slope -1/2 in a log-log AD plot) and "random walk" (slope +1/2 in a log-log AD plot) that are used in Kalibr. The noise model parameters can be determined directly from the Allan standard deviation.

$\sigma_{g}$ and $\sigma_{a}$ correspond to the values at $AD(\tau=1s)$ (point (1) in the figure below). This is only true since the noise power in most inertial sensors is dominated by "white noise" at a frequency of approximately 1Hz.

$\sigma_{b_g}$ and $\sigma_{b_a}$ are identified as the value of the (fitted) "random walk" diagonal at an integration time of $\tau=3s$ (point (2) in the figure below). This can be seen immediately when the Allan standard deviation is derived for a "random walk" process [1].




* * *

Figure 1: Allan standard deviation of a MEMS gyro with manually identified noise processes.

<img src="https://cloud.githubusercontent.com/assets/1916839/3589506/8f57d0ee-0c4e-11e4-9ab4-33821c040490.png" width="85%" align="middle"/>





## Kalibr IMU Noise Parameters in Practice

Some manufacturers provide an Allan standard deviation plot in the datasheet of the device. Otherwise, it needs to be computed from sensor data.
A useful open source tool for computing IMU parameters using Allan Deviation is [ori-drs/allan_variance_ros](https://github.com/ori-drs/allan_variance_ros) which seems to be actively maintained.
We recommend using this tool directly on ~15-24 hour dataset recording of the IMU being stationary.

<!--
- https://github.com/gaowenliang/imu_utils
- https://github.com/rpng/kalibr_allan
- https://github.com/AlbertoJaenal/imu_still_calibration
- https://github.com/Kyle-ak/imu_tk
- https://github.com/GAVLab/allan_variance
-->

_It is important_ to note that the IMU measurement error model used here is derived from a sensor which does not undergo motion, and at constant temperature. Hence scale factor errors and bias variation caused by temperature changes, for example, are not accounted for. So clearly, the model is optimistic. Particularly when using low-cost MEMS IMUs with Kalibr, you may have to increase the noise model parameters to "capture" these errors as well. In other words, if you use directly the "sigmas" obtained from static sensor data, Kalibr will tend to trust your IMU measurements too much, and its solution will not be optimal.

From our experience, for lowest-cost sensors, increasing the noise model parameters by a factor of 10x or more may be necessary. If you use Kalibr with such a device, please give us feedback, such that we can develop specific guidelines, device-specific parameter suggestions, or more advanced methods to determine these parameters.

***


# IMU Noise Model


$ E[n(t_1)n(t_2)] = \sigma^2 \delta(t_1 - t_2) $

where $ \delta(t_1 - t_2) $ is the Dirac delta function and $ \sigma^2 $ is the variance of the noise process.

The expression represents the autocorrelation function of the additive white noise process. Here's a breakdown of why $ E[n(t_1)n(t_2)] $ relates to the standard deviation:

1. White noise by definition has a constant power spectrum across all frequencies. In the time domain, this is reflected in its autocorrelation function: the noise values at different times are uncorrelated.

2. The expectation $ E[n(t_1)n(t_2)] $ quantifies how two noise values at times $ t_1 $ and $ t_2 $ are related. If the noise is white (i.e., purely random and uncorrelated), then the only non-zero expectation value is when $ t_1 = t_2 $.

3. The Dirac delta function $ \delta(t_1 - t_2) $ is a function that's zero everywhere except at $ t_1 = t_2 $, where it's "infinitely high" such that its integral over all time is 1. This ensures that the expectation is only non-zero when the two times are the same.

4. The factor $ \sigma^2 $ in front of the Dirac delta function is the variance of the noise process. The standard deviation of the noise is $ \sigma $, so $ \sigma^2 $ is the variance. The reason the variance appears here is that the autocorrelation function at zero time-lag (i.e., $ t_1 = t_2 $) represents the variance of the noise.

So, while $ E[n(t_1)n(t_2)] $ as a whole represents the autocorrelation function, the term $ \sigma^2 $ in the expression is indeed the variance of the noise process, and its square root would be the standard deviation.





 a discrete-time simulation of additive white noise in an Inertial Measurement Unit (IMU). Here's the breakdown of the equation and the terms involved:

$n_d[k] = \sigma_{g_d} w[k]$

Here:
- $n_d[k]$ represents the discrete-time white noise at a specific time index $k$.
- $w[k]$ is a standard normal random variable with a mean of $0$ and a variance of $1$, denoted as $w[k] \sim \mathcal{N}(0,1)$.
- $\sigma_{g_d}$ is the discrete-time noise standard deviation, calculated as $\sigma_{g_d} = \sigma_g \frac{1}{\sqrt{\Delta t}}$, where $\sigma_g$ is the continuous-time noise standard deviation, and $\Delta t$ is the sampling interval【7†source】.

The subscript $ g $ typically stands for "gyroscope" when discussing noise characteristics in the context of Inertial Measurement Units (IMUs). IMUs consist of a combination of sensors including accelerometers, gyroscopes, and sometimes magnetometers. Each of these sensors has its own noise characteristics. In the equations discussed, $ \sigma_g $ represents the standard deviation of the continuous-time white noise process associated with the gyroscope measurements. Similarly, $ \sigma_{g_d} $ represents the standard deviation of the discrete-time white noise process for the gyroscope. This notation helps differentiate the noise characteristics of the gyroscope from those of other sensors within the IMU.


Now, let's dive deeper into the components and related concepts:

1. **Discrete-Time White Noise**: In discrete-time, a white noise process is characterized by a sequence of uncorrelated random variables with zero mean and constant variance. This is in contrast to continuous-time white noise, which has issues due to the peculiarities of the continuum. The discrete-time white noise process is denoted by $w[k]$ in the equation and follows a standard normal distribution $w[k] \sim \mathcal{N}(0,1)$ [1](https://stats.stackexchange.com/questions/278263/infinite-variance-for-a-discrete-time-white-noise-process#:~:text=1,has%20the%20above%20three%20characteristics).

2. **IMU Noise Simulation**: IMUs are prone to various types of noise, including white noise. The simulation of IMU noise, especially in discrete time, helps in understanding and mitigating the effects of noise on sensor readings. This is crucial for applications like navigation and tracking, where precise sensor readings are vital [2](https://ch.mathworks.com/help/nav/ref/imu.html), [3](https://ch.mathworks.com/help/fusion/ug/introduction-to-simulating-imu-measurements.html).

3. **Noise Standard Deviation**: The term $\sigma_{g_d}$ in the equation represents the standard deviation of the discrete-time noise process. It's computed from the continuous-time noise standard deviation $\sigma_g$ and the sampling interval $\Delta t$, as $\sigma_{g_d} = \sigma_g \frac{1}{\sqrt{\Delta t}}$. This conversion is essential for simulating the continuous-time noise process in a discrete-time setting, aligning with the sampling rate of the IMU.



# The standard deviation of the discrete-time noise process

The standard deviation of the discrete-time noise process is derived from the continuous-time noise standard deviation by accounting for the sampling interval. This conversion is essential for simulating the continuous-time noise process in a discrete-time setting.

The formula to compute the standard deviation of the discrete-time noise process ($\sigma_{g_d}$) from the continuous-time noise standard deviation ($\sigma_g$) is given by:

$\sigma_{g_d} = \sigma_g \cdot \frac{1}{\sqrt{\Delta t}}$

where:
- $\sigma_{g_d}$ is the standard deviation of the discrete-time noise process.
- $\sigma_g$ is the continuous-time noise standard deviation.
- $\Delta t$ is the sampling interval (time between successive samples).

### Example:

Suppose we have a continuous-time noise standard deviation of $\sigma_g = 0.2$ rad/s, and our IMU is sampling at a rate of 100 Hz, which means the sampling interval is $\Delta t = 1/100 = 0.01$ seconds. We can use the formula to compute the standard deviation of the discrete-time noise process:

$\sigma_{g_d} = 0.2 \cdot \frac{1}{\sqrt{0.01}} = 0.2 \cdot 10 = 2$ rad/s





In [2]:
import math

# Given values
sigma_g = 0.2  # continuous-time noise standard deviation (rad/s)
sampling_rate = 100  # Hz
delta_t = 1 / sampling_rate  # sampling interval (s)

# Computing the standard deviation of the discrete-time noise process
sigma_g_d = sigma_g / math.sqrt(delta_t)
print(f'The standard deviation of the discrete-time noise process is {sigma_g_d} rad/s')

The standard deviation of the discrete-time noise process is 2.0 rad/s


In this code:
1. We first import the `math` module to use the `sqrt` function.
2. We then define the given values: the continuous-time noise standard deviation `sigma_g`, the sampling rate, and the sampling interval `delta_t`.
3. Finally, we compute the standard deviation of the discrete-time noise process `sigma_g_d` using the formula and print the result.



The standard deviation of continuous-time noise for an IMU, denoted as $\sigma_g$ for the gyroscope and possibly $\sigma_a$ for the accelerometer, is typically obtained through one of the following ways:

1. **Manufacturer Specifications:** 
   - IMU manufacturers often provide noise characteristics in their datasheets. This includes the standard deviation or other statistical measures of sensor noise.

2. **Experimental Measurement:**
   - Collect raw data from the IMU in a controlled environment, and compute the standard deviation of the noise.
   - One common method is the Allan Variance analysis, which helps determine various noise characteristics including the standard deviation of the white noise.

3. **Noise Analysis:**
   - By analyzing the sensor output data and comparing it against known references or ground truth, you can compute or estimate the standard deviation of the noise.

### Example:

Suppose you collect a set of gyroscope data over a period, with the IMU kept stationary to minimize motion-induced readings:

```python
import numpy as np

# Assume gyro_data is a numpy array containing your gyroscope readings
gyro_data = np.array([...])  # replace with your data

# Compute the standard deviation
sigma_g = np.std(gyro_data)

print(f'Standard deviation of continuous-time noise: {sigma_g}')
```

In this code snippet:
- We assume `gyro_data` is a numpy array containing your gyroscope readings.
- We use `np.std` to compute the standard deviation of these readings, which gives an estimate of $\sigma_g$.

### Note:
- The accuracy of $\sigma_g$ obtained through experimental measurement depends on the quality and duration of data collection, as well as the conditions under which the data is collected.
- If possible, it's always a good practice to cross-verify the obtained $\sigma_g$ with manufacturer specifications or other reputable sources.